In [3]:
import whisper
from utils import save_audio_from_mp4

In [4]:
model = whisper.load_model("base.en")      # turbo is recommended in sample code

100%|███████████████████████████████████████| 139M/139M [00:08<00:00, 17.7MiB/s]


In [5]:
# save_audio_from_mp4("Ses 03.mp4", audio_file="audio.mp3")

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


'audio.mp3'

In [6]:
result = model.transcribe("audio.mp3")

In [7]:
from dotenv import load_dotenv
from pathlib import Path
load_dotenv()

True

In [8]:
transcription = result['text']

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
messages = [
    SystemMessage(content="For a given transcription from a video to be published online, give a very short pre-read for the viewers on what to expect from the video"),
    HumanMessage(content=f"TRANSCRIPTION: {transcription}\nPREREAD:"),
]

response = model.invoke(messages)

In [ ]:
response.content

"In this video, we'll explore the concept of chain of thought prompting in language models, a technique that enhances their ability to perform multi-step reasoning tasks. You'll learn why traditional few-shot prompting might fall short in complex problem-solving scenarios, and how structured examples can guide models to generate more accurate answers by breaking down the reasoning process. Join us as we delve into the nuances of prompting strategies and their implications for improving language model performance!"

In [ ]:
messages = [
    SystemMessage(content="For a given transcription from the video published online, give a summary of the video for the viewers"),
    HumanMessage(content=f"TRANSCRIPTION: {transcription}\nSUMMARY:"),
]

response = model.invoke(messages)

In [ ]:
response.content

'In this video, the speaker discusses the concept of "chain of thought prompting" in large language models (LLMs). They explain that while few-shot prompting can be effective for certain tasks, it struggles with multi-step reasoning problems. The key idea is to encourage LLMs to generate intermediate thoughts rather than jumping directly to an answer. By providing detailed examples that illustrate the reasoning process, LLMs can learn to mimic this thinking style, leading to more accurate responses. The speaker emphasizes that the structure of the examples is crucial in guiding the model\'s thought process. They highlight the advantages of this method for solving complex problems, improving interpretability, and fostering reliable answers, especially in situations requiring intricate reasoning. The discussion underlines the significance of chain of thought prompting as a foundational technique in the realm of LLMs.'

In [13]:
messages = [
    SystemMessage(content="""You are an expert teacher in AI. For a given transcription from a video, section the content into 'Introduction', 'Learning objectives(if applicable)', 'Learning content', 'Challenges/Common mistakes(if applicable)', 'Conclusion'

Constraints:
- Do not alter the transcription given. Only put them into their relevant section"""),
    HumanMessage(content=f"TRANSCRIPTION: {transcription}\nSECTIONED CONTENT:"),
]

response = model.invoke(messages)

In [14]:
print(response.content)

**Introduction:**  
So, now, now what is the problem with few short prompting right? We just discussed right now that few short prompting we might be able to achieve lot of things. But when there is a multi step reasoning that is required right, we just prompting it by saying ok. So, here is a question here is answer. Now, I will give you the question you do the task ok. That is what few short prompting kind of you know let us you believe right. Now, this does not work when the reasoning that is required requires multiple steps right.

**Learning Content:**  
So, now you are just saying ok give some explanation and then just saying answer is 11 between this question to this answer there are so many steps and thoughts that are required to generate this right. So, now can we explicitly get the model to generate those thoughts and then answer the question rather than trying to jump to you know some number based on the context that is provided by the question right. That is the key idea be

In [17]:
messages = [
    SystemMessage(content="""You are an expert teacher in AI. For a given CONTENT from a video, generate 3 multiple choice questions from each section to recall from the content of the video to keep the users engaged
                  
Constrains:
- These sections are not visible to the user. Hence do not include these terms in the questions"""),
    HumanMessage(content=f"CONTENT: {response.content}\nQUESTIONS:"),
]
response = model.invoke(messages)

In [18]:
print(response.content)

**Introduction Questions:**

1. What is the primary limitation associated with few-shot prompting?
   - A) It requires too much data.
   - B) It struggles with multi-step reasoning.
   - C) It is too complex for models.
   - D) It generates irrelevant answers.

2. What is deemed essential for tasks that involve several steps?
   - A) Direct answers without explanations.
   - B) A single prompt example.
   - C) Detailed reasoning and thought processes.
   - D) Visual aids to assist understanding.

3. What belief does few-shot prompting foster among users regarding its capabilities?
   - A) Instantaneous results with no effort.
   - B) Simplified answers to complex questions.
   - C) Effective reasoning in multi-step problems.
   - D) Accurate predictions based on a single instance.

---

**Learning Content Questions:**

1. What is the aim of the "chain of thought" approach in language models?
   - A) To skip reasoning steps for faster answers.
   - B) To explicitly generate reasoning st